  # 0. Bibliotek och data
  Biblioteket `sqlite3` krävs för att skapa funktionen `Nyrad()` nedan. SLOSH-datafilen läses in.

In [1]:
import sqlite3
import pandas as pd

In [2]:
datafil = r'\\win.su.se\dfs\common\stressforskning.su.se\SLOSH-data\SLOSH och AMU\SLOSH\Enkätdata\SLOSH (Huvudfil, t.o.m. 2018)\Test\Slosh_all_amuar_18.xlsx'

df = pd.read_excel(datafil)

# 1. Definiera funktionen Nyrad
Funktionen `Nyrad()` tar (namnet på) en tabell, en tuple med kolumnnamn samt en tuple med kolumnvärden. Den skapar en rad av denna information och försöker lägga in den i databasen. Om den lyckas kommer den SQL-kod som används för att skapa raden att noteras i en text-fil. Denna text-fil ska tjäna dels som en backup men även som en logg. Skulle något hända med databasen går det enkelt att bara köra text-filen så återskapas allting.  
  Ifall funktionen inte lyckas lägga in en rad i databasen, kanske p.g.a. att "foregin key" inte respekteras, kommer ett felmeddelande konstatera detta. För mer information om hur SQLite används i python: https://pynative.com/python-sqlite-insert-into-table/.

In [196]:
def Nyrad(tabell, kolumner, värden):    
    try:
        # Koppla databasen
        conn = sqlite3.connect('SLOSH.db')

        # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
        cursor = conn.cursor()

        # Denna query ska skrivas i SQL och körs av python mot databasen
        sqlite_insert_query = 'INSERT INTO {} {} VALUES {}'.format(tabell, kolumner, värden)

        # Först måste 'foreign key constraint' slås på, i SQLite är detta avstängt som default. 
        cursor.execute('PRAGMA foreign_keys = 1')

        # Nu kan raden läggas in
        cursor.execute(sqlite_insert_query)

        # commit:a till databasen
        conn.commit()
        print('\033[1m' + 'Rad inlagd i tabellen {}'.format(tabell) + '\033[0m')

        # Stäng
        cursor.close()
        
        print('INSERT INTO {} {} VALUES {}'.format(tabell, kolumner, värden))
        
        # När en rad lagts till ska SQL-kommandot sparas i en textfil.
        text = open("SQL-logg.txt", "a", encoding = 'UTF-8')
        text.write('INSERT INTO {} {} VALUES {};\n \n'.format(tabell, kolumner, värden))
        text.close()

    except sqlite3.Error as error:
        print('\033[1m' + 'Det gick ej att infoga data: ' + '\033[0m', error)
    finally:
        if (conn):
            conn.close()

    

# 2.Transkribera  
Här transkriberas ett item i taget. Den transkriberade informationen kommer att användas för att skapa rader i flera olika tabeller. Exempelvis förekommer `kodlista` i tre olika tabeller. Se först hur stor andel av SLOSH som redan dokumenterats.    

**OBS!**  
Det är viktigt att databasfilen `SLOSH.db` inte används av något annat program samtidigt som pythonkärnan försöker få tillgång till den. Det går inte att skriva information till databasen från mer än ett ställe samtidigt. Se följande kommentar från stackoverflow:  
>Yes SQLite can support multiple users at once. It does however lock the whole database when writing, so if you have lots of concurrent writes it is not the database you want (usually the time the database is locked is a few milliseconds - so for most uses this does not matter). But it is very well tested and very stable (and widely used) so you can trust it.
You may read this short document for information when to use SQLite and not: http://www.sqlite.org/whentouse.html

## 2.1 Andel av SLOSH som redan har dokumenterats  
Här jämförs antalet dokumenterade variabler i relationsdatabasen med det totala antalet variabler i SLOSH-datafilen. Notera att siffran underskattas eftersom datafilen innehåller registervariabler samt `r_x`-variabler. Dessa kommer aldrig att läggas in i relationsdatabasen.

In [221]:
totvar = len(df.columns)

# Koppla databasen
conn = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = conn.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen
cursor.execute('SELECT variabel FROM Variabler')

dokvar = len(cursor.fetchall())

# Stäng
cursor.close()


print('\033[1m' + 'Andel dokumenterad data:' + '\033[0m' + '{}%'.format(round(100*dokvar/totvar, 2)))

Andel dokumenterad data:0.03%


## 2.2 working eller non-working? 
  Börja med att se efter ifall en given variabel förekommer i working, non-working eller båda.  
  Specificera först variabelnamnet.

In [222]:
variabel = 'sick1v_1'

In [223]:
r = 'r_{}'.format(variabel[-1])

frek_nw = df.loc[df[r] == 1, variabel].value_counts()

frek_w = df.loc[df[r] == 2, variabel].value_counts()

if len(frek_w) == 0:
    print('Variabeln {} förekommer ej i working'.format(variabel))
    
elif len(frek_nw) == 0:
    print('Variabeln {} förekommer ej i non-working'.format(variabel))
    
else:
    print('Resultat: \nVariabeln {} förekommer i working och i non-working'.format(variabel))

    

Resultat: 
Variabeln sick1v_1 förekommer i working och i non-working


## 2.3 Undersök frekvenstabellen och transkribera metadata

In [224]:
r = 'r_{}'.format(variabel[-1])

df.loc[df[r] == 2, variabel].value_counts(dropna = False).sort_index()

1.0    2606
2.0    1442
3.0     803
4.0     229
NaN      61
Name: sick1v_1, dtype: int64

Efter att ha tagit ställning till vilken (eller vilka) enkäter variabeln förekommer i går det att transkribera resten av informationen. Notera att ifall variabeln förekommer i både working och non-working måste det göras två insättningar relationsdatabasen. 

In [225]:
enkät = 'SLOSH 06 working'
beskrivning = ''
itemnr = 'B12_a'
enkättext = 'Hur många gånger har du varit sjukskriven en vecka eller mindre under de senaste 12 månaderna? Räkna inte med vård av sjukt barn i ditt svar.'
dubbelkodning = '1'

#kodsvar = {'1': 'Ja', '2': 'Nej', '.': 'Missing'}
kodsvar = {'1': 'Ingen gång', '2': 'En gång', '3': 'Två till tre gånger', '4': 'Fyra gånger eller mer', '.': 'Missing'}

## 2.4 Finns kodlistan redan?  
För att ta ställning till ifall kodlistan redan förekommer i databasen hämtas först all information i *Kodlistor_stor*. Ifall kodlistan redan finns sätts variabeln `kodlista` lika med det nummer kodlistan redan har i databasen. Om den *inte* finns sätts `kodlista` $ = $ 'NULL'. Ifall SQLite får detta som värde, och kolumnen ifråga är s.k. "AUTOINCREMENT", kommer kodlistan automatiskt få ett nytt heltal som id.

In [226]:
# Koppla databasen
conn = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = conn.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen.
cursor.execute('SELECT * FROM Kodlistor_stor')

# Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
rows = cursor.fetchall()

# Stäng databasen.
cursor.close()

kods = []
L = []

# gå igenom varje vektor i listan
for i in range(len(rows)):
    
    # Gäller bara första raden. Detta görs för att undvika index-fel nedan (i-1)
    if len(L) == 0:
        
        # Välj ut kod och svarstext.
        a = (rows[i][2], rows[i][3])
        
        # Lägg i a.
        L.append(a)
    else:
        
        # Ifall k_id skiljer från nuvarnade har loopen nått en ny kodlista. Skapa en dict av innehållet i L och
        # nollställ sedan L inför nästa kodlista.
        if rows[i][1] != rows[i-1][1]:
            d = {j[0]: j[1] for j in L}
            k = rows[i-1][1]
            D = [k, d]
            kods.append(D)
            L = []
        a = (rows[i][2], rows[i][3])
        L.append(a)

        
nykodlista = 'ja'
kodlista = 'NULL'

# Om kodlistan redan finns hämtas dess namn och läggs i variabeln kodlista.
for i in kods:
    if kodsvar == i[1]:
        kodlista = i[0]
        nykodlista = 'nej'
        break
        
        
print('Ny kodlista? \n{}'.format(nykodlista))    

IndexError: tuple index out of range

## 2.5 Finns svarstexterna redan?  
För att slippa att loopen som lägger in svarstexter nedan avbryts halvvägs igenom av att någon svarstext redan finns i databasen filtreras svarstexterna här. Om de redan är inlagda görs inget försök att lägga in dem igen.

In [203]:
# Koppla databasen
conn = sqlite3.connect('SLOSH.db')

# Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
cursor = conn.cursor()

# Denna query ska skrivas i SQL och körs av python mot databasen.
cursor.execute('SELECT * FROM Svarstexter')

# Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
rows = cursor.fetchall()

# Stäng databasen.
cursor.close()

# De transkriberade svarstexterna.
svarstexter = list(kodsvar.values())

# rows innehåller vektorer. Här plockat texten ut från vektorerna.
dbsvar = [i[0] for i in rows]

# En lista med de svarstexter som inte redan finns i databasen.
filtsvar = [i for i in svarstexter if i not in dbsvar]

filtsvar

['Ingen gång',
 'En gång',
 'Två till tre gånger',
 'Fyra gånger eller mer',
 'Missing']

# 4. Infoga rader i databasen  
Nu kommer den transkriberade informationen att användas tillsamman med funktionen `Nyrad()` för var och en av tabellerna i databasen.

## C: Kodlistor

In [204]:
tabell = 'Kodlistor'

kolumner = '(kodlista)'

värden = '({})'.format(kodlista)

In [205]:
if kodlista == 'NULL':
    Nyrad(tabell, kolumner, värden)

Rad inlagd i tabellen Kodlistor
INSERT INTO Kodlistor (kodlista) VALUES (NULL)


## G: Enkättexter

In [206]:
tabell = 'Enkättexter'

kolumner = '(enkättext)'

värden = '(\'{}\')'.format(enkättext)

In [207]:
Nyrad(tabell, kolumner, värden)

Rad inlagd i tabellen Enkättexter
INSERT INTO Enkättexter (enkättext) VALUES ('Hur många gånger har du varit sjukskriven en vecka eller mindre under de senaste 12 månaderna? Räkna inte med vård av sjukt barn i ditt svar.')


## H: Svarstexter

In [208]:
tabell = 'Svarstexter'

kolumner = '(svarstext)'

In [209]:
for i in filtsvar:
    värden = '(\'{}\')'.format(i)
    Nyrad(tabell, kolumner, värden)

Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('Ingen gång')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('En gång')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('Två till tre gånger')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('Fyra gånger eller mer')
Rad inlagd i tabellen Svarstexter
INSERT INTO Svarstexter (svarstext) VALUES ('Missing')


## D: Kodlistor_stor

In [210]:
tabell = 'Kodlistor_stor'

kolumner = ('KODLISTA', 'KOD', 'SVARSTEXT')

Ifall kodlistan inte tidigare fanns i databasen har den nu lagts längst ner/bak i *Kodlistor* och tilldelats ett nummer. Detta nummer (snarare än "NULL") ska nu användas för att lägga in information i *Kodlistor_stor*.

In [211]:
if kodlista == 'NULL':

    # Koppla databasen
    conn = sqlite3.connect('SLOSH.db')

    # Skapa en "cursor"... vet ej vad detta är, men det är nödvändigt.
    cursor = conn.cursor()

    # Denna query ska skrivas i SQL och körs av python mot databasen.
    cursor.execute('SELECT kodlista FROM Kodlistor')

    # Detta är en lista med vektorer. Varje vektor representerar en rad i tabellen.
    rows = cursor.fetchall()

    # Stäng databasen.
    cursor.close()

    kodlista = str(rows[-1][0])

In [212]:
kodlista

'1'

In [213]:
if nykodlista == 'ja':
    for i in kodsvar.keys():
        värden = '(\'{}\', \'{}\', \'{}\')'.format(kodlista, i, kodsvar[i])
        #värden = ('NULL', kodlista, i, kodsvar[i])
        Nyrad(tabell, kolumner, värden)

Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1', '1', 'Ingen gång')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1', '2', 'En gång')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1', '3', 'Två till tre gånger')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1', '4', 'Fyra gånger eller mer')
Rad inlagd i tabellen Kodlistor_stor
INSERT INTO Kodlistor_stor ('KODLISTA', 'KOD', 'SVARSTEXT') VALUES ('1', '.', 'Missing')


## B: Variabler

In [217]:
tabell = 'Variabler'

kolumner = ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA')

# värden = '(variabel, enkät, beskrivning, dubbelkodning, itemnr, enkättext, kodlista)

värden = '(\'{}\', \'{}\', \'{}\', \'{}\', \'{}\', \'{}\', \'{}\')'.format(variabel, enkät, beskrivning, 
                                                                           dubbelkodning, itemnr, enkättext,
                                                                          kodlista)

print('INSERT INTO {} {} VALUES {}'.format(tabell, kolumner, värden))

INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('sick1v_1', 'SLOSH 06 working', '', '1', 'B12_a', 'Hur många gånger har du varit sjukskriven en vecka eller mindre under de senaste 12 månaderna? Räkna inte med vård av sjukt barn i ditt svar.', '1')


In [220]:
Nyrad(tabell, kolumner, värden)

Rad inlagd i tabellen Variabler
INSERT INTO Variabler ('VARIABEL', 'enkät', 'BESKRIVNING', 'DUBBELKODNING', 'ITEMNR', 'enkättext', 'KODLISTA') VALUES ('sick1v_1', 'SLOSH 06 working', '', '1', 'B12_a', 'Hur många gånger har du varit sjukskriven en vecka eller mindre under de senaste 12 månaderna? Räkna inte med vård av sjukt barn i ditt svar.', '1')
